In [1]:
import sys, bz2, uuid, requests, json, pickle
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from joblib import dump, load

In [2]:
# defining the api-endpoint  
API_PREDICT_ENDPOINT = 'http://localhost:9090/api/predict'

# your API key here 
#API_KEY = "XXXXXXXXXXXXXXXXX"

In [3]:
iris = load_iris()
dataframe_load = pd.DataFrame(iris.data)
dataframe_load.columns = iris.feature_names 
data_label = iris.target
dataframe = dataframe_load.assign(LABEL=data_label)

In [4]:
_, dataframe_test = train_test_split(dataframe, test_size=0.1)
dataframe_test  = dataframe_test.reset_index(drop=True)

In [5]:
LABEL_COLUMN = 'LABEL'
columns = [LABEL_COLUMN]

In [6]:
X_test = dataframe_test.drop(columns, axis=1, inplace=False)
y_test = dataframe_test.filter(columns, axis=1)

In [7]:
# convert the dataframe to JSON
dataframe_json = X_test.to_json(orient='values')
print("dataframe_json:\n", dataframe_json)

dataframe_json:
 [[6.9,3.1,4.9,1.5],[5.0,3.4,1.5,0.2],[5.5,3.5,1.3,0.2],[5.0,3.5,1.3,0.3],[6.5,3.0,5.5,1.8],[6.3,2.3,4.4,1.3],[4.6,3.4,1.4,0.3],[5.6,2.9,3.6,1.3],[7.9,3.8,6.4,2.0],[4.8,3.4,1.6,0.2],[5.6,2.8,4.9,2.0],[4.7,3.2,1.3,0.2],[6.7,3.0,5.2,2.3],[6.9,3.1,5.4,2.1],[6.4,3.1,5.5,1.8]]


In [8]:
# send the request and get the results
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
data = {
  'dataframe_json' : dataframe_json
}
data_json = json.dumps(data)
req = requests.post(API_PREDICT_ENDPOINT, data=data_json, headers=headers)
#print(req, req.text)

predictions = json.loads(req.text)
print("predictions:\n", predictions)

predictions:
 "[1 0 0 0 2 1 0 1 2 0 2 0 2 2 2]"


In [9]:
y_pred = np.fromstring(predictions[2:-2], dtype=int, sep=' ')
y_pred

array([1, 0, 0, 0, 2, 1, 0, 1, 2, 0, 2, 0, 2, 2, 2])

In [10]:
acc = accuracy_score(y_true=y_test, y_pred=y_pred)
print("accuracy_score:", acc)

accuracy_score: 1.0
